# Configuration

In [2]:
# imports
import numpy as np
import pandas as pd
import seaborn as sea

from random import randrange

In [ ]:
# configure the libs
pd.set_option("display.max_columns", None)  #makes it display all columns in the data frame

# Algorithm Brief Explanation 

Decsion trees tend to be very sensitive to the dataset they were trained on. Meaning that if you take a dataset, train a decsion tree on it, and change it slighly. If you re-train the model, the tree produced may be very different from the one you originally started with. This means the model has high variance and may fail to generalize when given new data.

To solve this problem, Random Forest comes in. Random forest is an aggregate algorithm that uses muiltiple decsion trees to produce its calculation. 

1. Randomly select rows from the data set to create new datasets for each new tree created in the forest. Every dataset will contain the same number of rows as the original one. Using random sampling with replacement. This process is called bootstrapping. Bootstrapping ensures that we are not using the same data for every tree produced. This helps the model be less sensitive to the training data. 

2. Now each tree will be trained on its own data set independently. Each tree will not use every feature possible for training. Instead, a subset will be randomly selected for each tree only they will be used for training. This random feature selection helps reduce the corrolation between the trees. If you used every feature, then most of your trees will have the same decisions nodes and they will act very similarly. That would increase the variance. Another benifit is that some of the trees will be trained on less important features and so they will give bad predictions, but their will also be some trees that bad predictions in the opposite direction. So they will balance out.

3. Then build the trees.

4. Now to make the predictions. Given a new data point, run it through each tree and record each prediction. Whatever item has the majority vote is the item the model will report. This process of combining results from muiltiple models is called aggregation.

Note: Bootstraping + aggregation = bagging


What is the ideal size of the feature subset? For classification problems, research has found that values close to the log and sqrt of the total number of features work well.

So for example: 
num_features_for_split = sqrt(total_input_features)

# Implement the algorithm from Scratch

In [1]:
# imports needed
from random import seed
from random import randrange
from csv import reader
from math import sqrt

Modified split function from tutorial. Modified from decsion tree algorithm for random forest.

In [ ]:
# Select the best split point for the data
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0]) - 1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups}            
            

### Random forest implementation

In [ ]:
class random_forest:
    def __init__(self):
        
    #Split a dataset based on an attribute and an attribute value
    #TODO: more detail explination of purpose
    def test_split(self, index, value, dataset):
        left, right = list(), list()
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right
    
    def gini_index(self, groups, classes):
        # count all samples at split point
        n_instances = float(sum([len(group) for group in groups]))
        # sum weighted Gini index for each group
        gini = 0.0
        for group in groups:
            size = float(len(group))
            #avoid dividing by zero
            if size == 0:
                continue
            score = 0.0
            # score the group based on the score for each class
            for class_val in classes:
                p = [row[-1] for row in group].count(class_val) / size
                score += p * p
            # weight the group score by its relative size
            gini += (1.0 - score) * (size / n_instances) # gini formula
        return gini
    
    # Select the best split point for the data
    def get_split(dataset, n_features):
        class_values = list(set(row[-1] for row in dataset))
        b_index, b_value, b_score, b_groups = 999, 999, 999, None
        features = list()
        while len(features) < n_features:
            index = randrange(len(dataset[0]) - 1)
            if index not in features:
                features.append(index)
        for index in features:
            for row in dataset:
                groups = test_split(index, row[index], dataset)
                gini = gini_index(groups, class_values)
                if gini < b_score:
                    b_index, b_value, b_score, b_groups = index, row[index], gini, groups
        return {'index': b_index, 'value': b_value, 'groups': b_groups}
    
    # Create a terminal(leaf) node
    def to_terminal(self, group):
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key = outcomes.count)

    # Create child splits for a node or make terminal
    def split(self, node, max_depth, min_size, n_features, depth):
        left, right = node['groups']
        del(node['groups'])
        # check for a no split
        if not left or not right:
            node['left'] = node['right'] = self.to_terminal(left + right)
            return
        
        # Check for max depth
        if depth >= max_depth:
            node['left'], node['right'] = self.to_terminal(left), self.to_terminal(right)
            return
        
        # process left child
        if len(left) <= min_size:
            node['left'] = self.to_terminal(left)
        else:
            node['left'] = self.get_split(left, n_features)
            self.split(node['left'], max_depth, min_size, n_features, depth + 1)
            
        # process right child
        if len(right) <= min_size:
            node['right'] = self.to_terminal(right)
        else:
            node['right'] = self.get_split(right, n_features)
            self.split(node['right'], max_depth, min_size, n_features, depth + 1)
            
    # Build a decsion tree
    def build_tree(self, train, max_depth, min_size, n_features):
        root = self.get_split(train, n_features)
        self.split(root, max_depth, min_size, n_features, 1)
        return root
    
    # Make a prediction with a decision tree
    def predict(self, node, row):
        if row[node['index']] < node['value']:
            return self.predict(node['left'], row)
        else:
            return node['left']
        
        
        
    

# Implement the algorithm from library 